In [18]:
from datetime import datetime, timedelta as td
import numpy as np
import pandas as pd
import os
import sys

dtf = datetime.strftime
dtp = datetime.strptime

In [19]:
class ExpLogHandler:
    def __init__(self, log_names):
        self.log_names = log_names
        
    def getData(self):
        self.data = pd.read_csv(self.log_names[0], error_bad_lines=False)
        for i in range(1, len(self.log_names)):
            data_next   = pd.read_csv(self.log_names[i], error_bad_lines=False)
            data_frames = [self.data, data_next]
            self.data   = pd.concat(data_frames, ignore_index=True)
        self.data.drop(self.data.loc[(self.data["Timestamp"] == "Timestamp")].index.tolist())
        self.dataLength = self.data.shape[0]

        wrchckbrdLocs  = self.data.loc[(self.data["Board"] == "M") & (self.data["Mikroe_socket"] == "A") &
                                       (self.data["Status"] == "WRCHCKBRD")].index.tolist()
        wrchckbrdIndex = [(i, "WRCHCKBRD") for i in wrchckbrdLocs]
        initLocs       = self.data.loc[(self.data["Board"] == "M") & (self.data["Mikroe_socket"] == "A") &
                         (self.data["Status"] == "INIT")].index.tolist()
        initIndex      = [(i, "INIT") for i in initLocs]
        runIndex       = wrchckbrdIndex + initIndex
        runIndex       = sorted(runIndex)
        runIndex.append((self.dataLength, "END"))
        
        runId  = 1
        runCol = [''] * self.dataLength
        for h in range(len(runIndex) - 1):
            if runIndex[h][1] == "WRCHCKBRD":
                start              = runIndex[h][0]
                end                = runIndex[h+1][0]
                runCol[start:end]  = [runId]*(end-start)
                runId              += 1

        self.data.insert(loc=0, column="Run_ID", value=runCol)
        return self.data

In [20]:
class BeamLogHandler:
    def __init__(self, firstTime, lastTime, targetDir_fullpathList):     
        self.beamLogFnameFormat          = "%Y-%m-%d"
        self.firstTime                   = firstTime
        self.lastTime                    = lastTime
        self.targetDir_fullpathList      = targetDir_fullpathList
        self.targetDir_fullpathList[-4:] = ["neutrons"]

    def getLogs(self):
        firstBeamTime = dtf(self.firstTime, self.beamLogFnameFormat)
        beamlogDayDif = (self.lastTime - self.firstTime).days
        beamPath      = "../../../neutrons/countlog-"
        self.beamlog  = pd.read_csv(f"{beamPath}{firstBeamTime}.txt",
                        delim_whitespace=True, header=None, skiprows=1)
        nextTime      = self.firstTime
        for i in range(0, beamlogDayDif+1):
            nextTime      = nextTime + td(days=1)
            nextBeamTime  = dtf(nextTime, self.beamLogFnameFormat)
            try:
                nextBeamlog   = pd.read_csv(f"{beamPath}{nextBeamTime}.txt",delim_whitespace=True,header=None,skiprows=1)
            except FileNotFoundError:
                pass
            beamlogFrames = [self.beamlog, nextBeamlog]
            self.beamlog  = pd.concat(beamlogFrames, ignore_index=True)
            
        self.beamlog.columns = ["Date","HMS_time","Millisecs","Count1","Count2",
                                "Count3","Count4","protonCharge","Beam_current"]
        return self.beamlog

In [21]:
class Run:
    def __init__(self, name, data):
        self.name       = name
        self.valid      = -1
        self.hot        = -1
        self.delay      = -1
        self.num        = [int(s) for s in name.split('_') if s.isdigit()][0]
        self.df         = data.loc[(data['Run_ID'] == self.num)]
        self.errors    = -1
        self.errorList = []

In [22]:
class Analyser:
    def __init__(self, targetDir_fullpath):
        self.targetDir_fullpath  = targetDir_fullpath
        self.expLogTstampFormat  = "%Y-%m-%d_%H-%M-%S-%f"
        self.beamLogTstampFormat = "%d/%m/%Y %H:%M:%S"
 
    def setup(self):
        targetDir_fullpathList = self.targetDir_fullpath.split("/")
        targetDir_trunc        = targetDir_fullpathList[-3:]
        self.chip              = targetDir_trunc[0]
        self.variant           = targetDir_trunc[1]
        self.size              = targetDir_trunc[2]

        # Get data from logs
        self.log_names     = [self.targetDir_fullpath+i for i in os.listdir(self.targetDir_fullpath) if ".csv" in i]
        self.log_names.sort(key=os.path.getmtime)
        self.ExpLogHandler = ExpLogHandler(self.log_names)
        self.data          = self.ExpLogHandler.getData()

        # Get beam status from logs
        firstTimestamp      = self.data[0:1]['Timestamp'][0]
        lastTimestamp       = self.data[self.data.shape[0]-2:self.data.shape[0]-1]['Timestamp'][self.data.shape[0]-2]
        firstTime           = dtp(firstTimestamp, self.expLogTstampFormat)
        lastTime            = dtp(lastTimestamp, self.expLogTstampFormat)
        self.BeamLogHandler = BeamLogHandler(firstTime, lastTime, targetDir_fullpathList)
        self.beamlog        = self.BeamLogHandler.getLogs()

        # Create datetime objects from beamlog timestamps
        beamlogTstamps    = (self.beamlog['Date']+self.beamlog['HMS_time']+self.beamlog['Millisecs'].apply(str)).tolist()
        self.beamlogTimes = []
        for i in beamlogTstamps:
            if len(i) > 23:
                i = i[0:26]
            self.beamlogTimes.append(i)
        self.beamlogTimes = [dtp(i, '%d/%m/%Y%H:%M:%S0.%f') for i in self.beamlogTimes]

    def beamOn(self, firstTstamp, lastTstamp):
        bOfirstTime = firstTstamp
        bOlastTime = lastTstamp
        beamTimeNearFirstTime = min([i for i in self.beamlogTimes if i <= bOfirstTime], key=lambda x: abs(x - bOfirstTime))
        beamTimeNearLastTime  = min([i for i in self.beamlogTimes if i >= bOlastTime], key=lambda x: abs(x - bOlastTime))
        firstRow              = self.beamlog.loc[(self.beamlog['Date'] == dtf(beamTimeNearFirstTime, '%d/%m/%Y')) &
                                (self.beamlog['HMS_time'] == dtf(beamTimeNearFirstTime, '%H:%M:%S'))]
        lastRow               = self.beamlog.loc[(self.beamlog['Date'] == dtf(beamTimeNearLastTime, '%d/%m/%Y')) & 
                                (self.beamlog['HMS_time'] == dtf(beamTimeNearLastTime, '%H:%M:%S'))]
        count4Dif             = lastRow.iloc[0]['Count4'] - firstRow.iloc[0]['Count4']
        numRows               = lastRow.index.astype(int)[0] - firstRow.index.astype(int)[0]
        cps                   = count4Dif / numRows
        if cps > 1:
            return 1
        return 0
 
    def createRuns(self):
        runNames  = []
        self.runs = {}
        for i in range(1, self.data[self.data['Run_ID'].last_valid_index(): \
                          self.data['Run_ID'].last_valid_index()+1]['Run_ID'][self.data['Run_ID'].last_valid_index()]+1):
            runNames.append(f"run_{i}")
        for i in runNames:
            self.runs[i] = Run(i, self.data)

    def processRuns(self): 
        for value in self.runs.values():
            try:
                before_delay_i  = value.df.loc[(value.df['Mikroe_socket']=='D') & (value.df['Status']=='STORE_OK')].index[0]
                before_delay_t  = value.df.loc[before_delay_i]['Timestamp']
                after_delay_i   = value.df.loc[(value.df['Mikroe_socket']=='A') & (value.df['Status']=='VERIF')].index[0]+1
                after_delay_t   = value.df.loc[after_delay_i]['Timestamp']
                before_delay_dt = dtp(before_delay_t, self.expLogTstampFormat)
                after_delay_dt  = dtp(after_delay_t, self.expLogTstampFormat)   
                dif             = after_delay_dt - before_delay_dt
                if td(seconds = 0.1) < dif < td(seconds = 0.5):
                    value.delay = 0.1
                if td(seconds = 0.5) < dif < td(seconds = 5):
                    value.delay = 1
                if td(seconds = 5) < dif < td(seconds = 50):
                    value.delay = 10
                if td(seconds = 50) < dif < td(seconds = 500):
                    value.delay = 100
                if td(seconds = 500) < dif < td(seconds = 5000):
                    value.delay = 1000

                try:
                    if self.beamOn(before_delay_dt, after_delay_dt):
                        value.hot = 1
                    else:
                        value.hot = 0
                except ValueError:
                    print(value.name)
                    
                try:
                    value.errors = value.df['Status'].value_counts()['SDC']
                except KeyError:
                    value.errors = 0

                if value.errors:
                    value.errorList = list(zip(value.df.loc[value.df['Status'] == 'SDC']['SDC_val'],
                                               value.df.loc[value.df['Status'] == 'SDC']['SDC_loc']))
            except IndexError:
                value.valid = 0
            else:
                value.valid = 1

In [23]:
sram3_1_half = Analyser("C:/Users/Sujit/Documents/STFC/Code/Mikroe/nvRAM_experiment/results/live/SRAM3/1/half/")

In [24]:
sram3_1_half.setup()

In [25]:
sram3_1_half.createRuns()

In [26]:
sram3_1_half.processRuns()

In [36]:
ranks = []
for run in sram3_1_half.runs.values():
    if run.valid & run.hot & run.errors:
        ranks.append((run.errors, run.num))
        if all(flag == 0 for (flag, _) in run.errorList):
            print(run.num)
        if all(flag == 0xFF for (flag, _) in run.errorList):
            print(run.num)

ranks.sort(reverse=True)

I have a list of all the errors that occur in a particular run, and I have a list of runs sorted by number of errors in 
descending order. The goal of looking at the data is to have three totals: total number of true SEEs, total number of row 
errors, and total number of full array errors. The code therefore must distinguish between full errors, row errors, and
must find any true SEEs hidden within a row error. 

Find numeric difference between each memory location of entire error list. For row arrays, there will be many of the same difference, i.e. constant integer difference between memory locs for row errors. Highly likely to affect whole array so probably most of the errors will be adhering to that pattern, if a row error happened. Can identify indices of errors which didn't follow the pattern and they must be SEEs. IMPORTANT: row errors always have the same error byte. If there was an error at a location that followed a pattern but with a different error byte, then it means that an SEE happened at that loc AFTER (or during) the row error - count it also as an SEE.

In [55]:
ranks[0:15]

[(136297, 1607),
 (134231, 1359),
 (133401, 1505),
 (131067, 448),
 (123063, 553),
 (73681, 623),
 (73085, 1581),
 (67591, 648),
 (63825, 867),
 (59205, 901),
 (24363, 443),
 (13267, 1541),
 (12679, 1506),
 (7767, 1080),
 (7025, 1397)]

In [59]:
sram3_1_half.runs["run_1397"].df[7010:7060]

,Run_ID,Timestamp,Board,Mikroe_socket,Status,SDC_val,SDC_loc
1700539,1397,2021-06-05_02-12-17-252162,M,D,SDC,8A,1DB9B
1700540,1397,2021-06-05_02-12-17-268176,M,D,SDC,AE,1DBF6
1700541,1397,2021-06-05_02-12-17-284200,M,D,SDC,A2,1DBFB
1700542,1397,2021-06-05_02-12-17-300235,M,D,SDC,A2,1DC7B
1700543,1397,2021-06-05_02-12-17-316210,M,D,SDC,8A,1DCE1
1700544,1397,2021-06-05_02-12-17-332283,M,D,SDC,A2,1DCFB
1700545,1397,2021-06-05_02-12-17-364031,M,D,SDC,A2,1DD7B
1700546,1397,2021-06-05_02-12-17-380086,M,D,SDC,A2,1DDFB
1700547,1397,2021-06-05_02-12-17-396202,M,D,SDC,A2,1DE7B
1700548,1397,2021-06-05_02-12-17-412240,M,D,SDC,A2,1DEFB


In [48]:
all(x == sram3_1_half.runs["run_1607"].errorList[0][0] for x in sram3_1_half.runs["run_1607"].errorList)

False

In [54]:
for x in sram3_1_half.runs["run_1607"].errorList:
    if x[0] == sram3_1_half.runs["run_1607"].errorList[0][0]:
        continue
    else:
        print("Not same")
        break
print("all same")

Not same
all same


In [ ]:
test_list = sram3_1_half.runs["run_1020"].errorList
for i in range(0, len(test_list)):
    if test_list[i+1][1][-1] == test_list[i][1][-1]:
        print("match")
    else:
        print("no match")